In [1]:
%matplotlib inline
# %matplotlib notebook
import rasterio
from rasterio.coords import BoundingBox
from shapely.geometry import box
import geopandas as gpd
from rasterio.mask import mask
import os
import pandas as pd
import numpy as np

In [2]:
url_imagen = '/home/diego/Proyecto Vivero/data/01_VIVERO/01_imagenes/imagen_mesones_rgb.tif'
shp_puntos = '/home/diego/Proyecto Vivero/data/01_VIVERO/02_puntos_plantas/plantas_mesones_2.shp'
shp_bandejas = '/home/diego/Proyecto Vivero/data/bandejas.shp'

In [3]:
puntos = gpd.read_file(shp_puntos)
bandejas = gpd.read_file(shp_bandejas)
raster = rasterio.open(url_imagen)
cuentas = pd.DataFrame(columns=['imagen', 'cant_arboles'])

In [4]:
bandejas

,Id,geometry
0,0,"POLYGON ((738103.2334920149 6090370.06029545, ..."
1,0,"POLYGON ((738103.5159373647 6090370.543448572,..."
2,0,"POLYGON ((738103.7985446779 6090371.02687875, ..."
3,0,"POLYGON ((738104.0811520088 6090371.510308959,..."
4,0,"POLYGON ((738104.3786079967 6090372.01913939, ..."
5,0,"POLYGON ((738103.5785957333 6090369.858049181,..."
6,0,"POLYGON ((738103.8611255893 6090370.341346824,..."
7,0,"POLYGON ((738104.143733629 6090370.824778211, ..."
8,0,"POLYGON ((738104.4263416864 6090371.308209625,..."
9,0,"POLYGON ((738104.7239416484 6090371.817286301,..."


In [91]:
pl = raster.index(bandejas["geometry"][0].bounds[0],bandejas["geometry"][0].bounds[1])
pt = raster.index(bandejas["geometry"][0].bounds[0],bandejas["geometry"][0].bounds[3])
pr = raster.index(bandejas["geometry"][0].bounds[2],bandejas["geometry"][0].bounds[3])
width = pr[1]-pt[1]
height = pl[0]-pt[0]
print(width, height, pt, pl)

55 61 (1109, 1679) (1170, 1679)


In [7]:
%%time
contador = 0
idx = 0
while (1):
    #transformacion de la imagen
#     try:
    out_img, out_transform = mask(raster, shapes=[bandejas["geometry"][contador]], crop=True)
    out_meta = raster.meta.copy()
    out_meta.update(
    {"driver": "GTiff",
          "height": 55,
          "width": 61,
          "transform": out_transform,
          "crs": raster.crs})
    with rasterio.open("/home/diego/Proyecto Vivero/train/bandeja"+str(contador)+".tif", "w", **out_meta) as dest:
        dest.write(out_img)

#interseccion ptos
    cantidad=puntos.intersects(bandejas["geometry"][contador]).value_counts()
    if (True in cantidad.index):
        numero = cantidad[True]
    else:
        numero = 0
    cuentas.loc[contador] = ["bandeja"+str(contador)+".tif",numero]
#     except:
#         print(contador)
#         break
    contador = contador + 1   

KeyError: 515

In [8]:
cuentas

,imagen,cant_arboles
0,bandeja0.tif,64
1,bandeja1.tif,74
2,bandeja2.tif,68
3,bandeja3.tif,66
4,bandeja4.tif,77
5,bandeja5.tif,77
6,bandeja6.tif,72
7,bandeja7.tif,68
8,bandeja8.tif,74
9,bandeja9.tif,86


In [9]:
cuentas.to_csv("/home/diego/Proyecto Vivero/train/cuentas.csv")